In [ ]:
"""
종목조회 기본데이터 만듦
"""
import requests
import json
import time
import math
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# 함수정의 #######################################################################

def writeFile(fileName, data):
    f = open(fileName, 'w')
    f.write(json.dumps(data))
    f.close()
    
# 일자별시세 --------------------------------------
def getDays30(iscd):
    print("일자별시세###################")
    headersDaily = { 
        "content-type":"application/json",
        'appKey': appKey,
        'appSecret': appSecret,
        'authorization': 'Bearer ' + accessToken,
        'tr_id':'FHKST01010400',
        'tr_cont':'',
        'custtype':'P',
        #'mac_address':macAddress
        }
    params = {
        'FID_COND_MRKT_DIV_CODE':'J',
        'FID_INPUT_ISCD': iscd,
        'FID_ORG_ADJ_PRC':'0',
        'FID_PERIOD_DIV_CODE':'D'
    }
    sendUrl = url +'/uapi/domestic-stock/v1/quotations/inquire-daily-price'
    res = requests.get(sendUrl, headers=headersDaily, params=params)
    #print(res)
    #print(res.headers)
    #print(res.json())
    output = res.json()['output']
    return output

# 기간별시세 --------------------------------------
def getPeriod(iscd, start, end):
    print("기간별시세###################")
    headersDaily = { 
        "content-type":"application/json",
        'appKey': appKey,
        'appSecret': appSecret,
        'authorization': 'Bearer ' + accessToken,
        'tr_id':'FHKST03010100',
        'tr_cont':'',
        'custtype':'P',
        #'mac_address':macAddress

        }
    params = {
        'FID_COND_MRKT_DIV_CODE':'J',
        'FID_INPUT_ISCD': iscd,
        'FID_INPUT_DATE_1':start,
        'FID_INPUT_DATE_2':end,
        'FID_PERIOD_DIV_CODE':'D',
        'FID_ORG_ADJ_PRC':'0'
    }
    sendUrl = url +'/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice'
    res = requests.get(sendUrl, headers=headersDaily, params=params)
    #print(res)
    #print(res.headers)
    #print(res.json())
    output = res.json()['output2']

    return output

# HashKey
def getHashKey(datas):
    sendUrl = url + '/uapi/hashkey'
    #print(sendUrl)
    res = requests.post(sendUrl, headers=headers, data=json.dumps(datas))
    #print(res)
    #print(res.json()['HASH'])
    hashKey = res.json()['HASH']
    #print('hashKey', hashKey);
    return hashKey

# 계좌번호조회 ----------------------------------
def getAcntList(CTX_AREA_FK100, CTX_AREA_NK100):
    print("잔고조회##############")
    headersDaily = { 
        "content-type":"application/json",
        'appKey': appKey,
        'appSecret': appSecret,
        'authorization': 'Bearer ' + accessToken,
        'tr_id':'TTTC8434R',
        'tr_cont':'',
        'custtype':'P',
        'hashkey': hashKey
        #'mac_address':macAddress

        }
    params = {
        'CANO':acntNo,
        'ACNT_PRDT_CD':'01',
        'AFHR_FLPR_YN':'N',
        'OFL_YN':'',
        'INQR_DVSN':'02',
        'UNPR_DVSN':'01',
        'FUND_STTL_ICLD_YN':'N',
        'FNCG_AMT_AUTO_RDPT_YN':'N',
        'PRCS_DVSN':'00',
        'CTX_AREA_FK100':CTX_AREA_FK100,
        'CTX_AREA_NK100':CTX_AREA_NK100
    }
    sendUrl = url +'/uapi/domestic-stock/v1/trading/inquire-balance'
    res = requests.get(sendUrl, headers=headersDaily, params=params)
    return res

# 스토케스틱
def getStochestic(val, hVal, lVal):

    chart_history = []
    chart_h_history = []
    chart_l_history = []
    chart_max_history = []
    chart_min_history = []
    #try:

    for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]:
        chart_history.append(val[index:index + 5])
        chart_h_history.append(hVal[index:index + 5])
        chart_l_history.append(lVal[index:index + 5])
        #print(chart_h_history[index])
        chart_max_history.append(np.max(chart_h_history[index]))
        chart_min_history.append(np.min(chart_l_history[index]))


    # 스토캐스틱 %K (fast %K) = (현재가격-N일중 최저가)/(N일중 최고가-N일중 최저가) ×100
    st_kf_history = []
    for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                    26, 27, 28, 29, 30, 31, 32, 33, 34]:
        # print('***', chart_history[index], chart_min_history[index], chart_max_history[index])
        # print(index, '***', chart_history[index][0], chart_min_history[index],
        #      chart_max_history[index], (chart_history[index][0] - chart_min_history[index]) / (
        #                chart_max_history[index] - chart_min_history[index]) * 100)
        st_kf_history.append(
            (chart_history[index][0] - chart_min_history[index]) / (
                    chart_max_history[index] - chart_min_history[index]) * 100)

    # print('st_kf_history', st_kf_history)
    # 스토캐스틱 %D (fast %D) = m일 동안 %K 평균 = Slow %K
    st_k_history = []
    st_d_history = []
    for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                    26, 27, 28, 29, 30, 31, 32, 33, 34]:
        st_k_history.append(np.average(st_kf_history[index:index + 3]))
    
    #print('st_k_history', st_k_history)
    # slow %D = t일 동안의 slow %K 평균
    #for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
    #                26, 27, 28, 29, 30, 31, 32, 33, 34]:
    #    st_d_history.append(np.average(st_k_history[index:index + 15]))
        # print('st_d_history', st_d_history)
    #except:
    #    print("Unexpected error:", sys.exc_info())
    return st_k_history[0]

#######################################################################

In [ ]:
file = open('acntNo.txt', 'r')
acntNo = file.read()
file.close()

file = open('appKey.txt', 'r')
appKey = file.read()
file.close()

file = open('appSecret.txt', 'r')
appSecret = file.read()
file.close()

hashKey = ''
headers = { "content-type":"application/json",
'appKey': appKey,
'appSecret': appSecret}

# 인증키
body = {"grant_type":"client_credentials",
        "appkey":appKey, 
        "appsecret":appSecret}

sendUrl = url + '/oauth2/tokenP'
res = requests.post(sendUrl, headers=headers, data=json.dumps(body))
#print(res)
#print(res.json())
accessToken = res.json()['access_token']

In [ ]:
jongTarget = [
    { 'num':	1	,'iscd':'005930', 'name':'삼성전자' },
    { 'num':	2	,'iscd':'000270', 'name':'기아'}]
jong = []
order = []
url = 'https://openapi.koreainvestment.com:9443'

#file = open('acntNo.txt', 'r')
#acntNo = file.read()
#file.close()


dtNow = datetime.now()
dtNowStr = str(dtNow.date()).replace('-','')
dtEnd = str(dtNow.date()).replace('-','')
dtStart = dtNow + timedelta(days=-90)
dtStart = str(dtStart.date()).replace('-','')

print(dtStart + '~' + dtEnd)

inputDate = input('다른 날짜를 입력하시겠습니까? C to Continue, Q to quit, date = date   >>> ')

if inputDate == 'C'  or inputDate == 'c' or inputDate == '':
    print("continue")
elif inputDate == 'Q'  or inputDate == 'q':
    quit()
else:
    dtEnd = inputDate
    print(dtStart + '~' + dtEnd)
    inputTemp = ('any key to continue...')

jongTemp = []

jongAll = []

for item in jongTarget:
    existJongTemp = False
    for item2 in jongTemp:
        if item['iscd'] == item2['iscd']:
            item['price'] = item2['price']
            item['amto'] = item2['amto']
            item['amt'] = item2['amt']
            item['profit'] = item2['profit']
            item['qty'] = item2['qty']
            existJongTemp = True
    jongAll.append(item)

for item in jongAll:
    if item.get('qty'):
        item['exist'] = True
    else: 
        item['qty'] = 0
        item['exist'] = False


# 종목별 일자별 시세
for index, item in enumerate(jongAll):
    chartValue = []
    chartHigh = []
    chartLow = []
    chartDate = []
    time.sleep(0.3)
    #print(str(item['num'])+ ': ' + item['name'])
    #print(item)
    chartData = getPeriod(item['iscd'], dtStart, dtEnd)
    #chartData = getDays30(item['iscd'])
    for item2 in chartData:
        #print('>>>',item2)
        chartDate.append(item2['stck_bsop_date'])
        chartValue.append(int(item2['stck_clpr']))
        chartHigh.append(int(item2['stck_hgpr']))
        chartLow.append(int(item2['stck_lwpr']))
    


    item['rate'] = int((int(chartValue[0])-int(chartValue[1]))/int(chartValue[1]) * 1000)/1000
    item['price'] = int(chartValue[0])

    # stockestc 호출 
    bfChartValue = chartValue[1::]
    bfChartHigh = chartHigh[1::]
    bfChartLow = chartLow[1::]
    #print ('test', chartValue, bfChartValue)
    item['st'] = int(getStochestic(chartValue, chartHigh, chartLow) * 1000)/1000
    item['stBf'] = int(getStochestic(bfChartValue, bfChartHigh, bfChartLow) * 1000)/1000
    item['updateDate'] = dtEnd

for index, item in enumerate(jongAll):
    print(item['name'] + ' ' + str(item['st']) + ' ' + str(item['rate']) + ' ' + item['updateDate'])